In [ ]:
#to do
check to see why we need to pass an attention mask when calling model.generate for the pythia model.
maybe we also need to pass an attention mask to pythia (and phi) when training on it's internal representations.

In [1]:
!pip install git+https://github.com/bigscience-workshop/promptsource.git

/home/ubuntu/mambafs/mamba-hidden-states


In [1]:
%%bash
python -m venv mamba_env
source mamba_env/bin/activate

#need this for lambda stack
#for cudnn_so in /usr/lib/python3/dist-packages/tensorflow/libcudnn*; do
#  sudo ln -s "$cudnn_so" /usr/lib/x86_64-linux-gnu/
#done

pip install packaging wheel torch ipykernel datasets
pip install accelerate -U
#assume your in mamba directory
pip install .

#install bigcode eval harness
cd 3rdparty
git clone https://github.com/bigcode-project/bigcode-evaluation-harness.git
cd ..
pip install -e 3rdparty/bigcode-evaluation-harness

#install lm eleuther ai eval harness
git submodule update --init --recursive
pip install -e 3rdparty/lm-evaluation-harness
pip install git+https://github.com/bigscience-workshop/promptsource.git
#install flash attention for phi model
pip install flash_attn

#install environment as a ipykernel for use in jupyter notebook
python -m ipykernel install --user --name=mamba_env

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 684.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.1/114.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 41.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00ta 0:00:01



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Processing /home/mamba-hidden-states
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.5/446.5 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.7 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.4

...

  Resolved https://github.com/bigscience-workshop/promptsource.git to commit 7dab96a3eeb3717cea633705135ebc488885d709
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 31.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 36.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.2 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.0 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for flash_attn: filename=flash_attn-2.4.2-cp310-cp310-linux_x86_64.whl size=113714653 sha256=560df8869000f20f44af8ff42721c3d08ccfe5624bdc3c00e056a783e2a54fb9
  Stored in directory: /home/.cache/pip/wheels/9d/cf/7f/d14555553b5b30698dae0a4159fdd058157e7021cec565ecaa
Successfully built flash_attn



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Installed kernelspec mamba_env in /home/.local/share/jupyter/kernels/mamba_env


# Eval on squad 2

In [1]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model mamba \
    --model_args pretrained=state-spaces/mamba-130m \
    --tasks lambada_openai --device cuda:0 --batch_size 64


2024-01-12:22:20:15,536 INFO     [utils.py:148] Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-12:22:20:15,537 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-12:22:20:16,420 INFO     [config.py:58] PyTorch version 2.1.2 available.
2024-01-12:22:20:31,619 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for gsmk boolq, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:20:31,622 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for EleutherAI/asdiv, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:20:35,679 INFO     [__main__.py:184] Selected Tasks: ['lambada_openai']
config.json: 100%|██████████| 199/199 [00:00<00:00, 401kB/s]
pytorch_model.bin: 100%|██████████| 517M/517M [00:06<00:00, 85.2MB/s] 
tokenizer_config.json: 100%|██████████| 156/156 [00:00<00

bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:10<00:00,  9.54it/s]


mamba (pretrained=state-spaces/mamba-130m), limit: None, num_fewshot: None, batch_size: 64
|    Tasks     |Version|Filter|  Metric  | Value |   |Stderr|
|--------------|-------|------|----------|------:|---|-----:|
|lambada_openai|Yaml   |none  |perplexity|16.0459|±  |0.5093|
|              |       |none  |acc       | 0.4425|±  |0.0069|



In [2]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model hf \
    --model_args pretrained=EleutherAI/pythia-410m,revision=step100000,dtype="float" \
    --tasks lambada_openai --device cuda --batch_size 64

2024-01-12:22:22:27,974 INFO     [utils.py:148] Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-12:22:22:27,975 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-12:22:22:28,790 INFO     [config.py:58] PyTorch version 2.1.2 available.
2024-01-12:22:22:39,225 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for gsmk boolq, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:22:39,227 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for EleutherAI/asdiv, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:22:42,449 INFO     [__main__.py:184] Selected Tasks: ['lambada_openai']
2024-01-12:22:22:42,466 INFO     [huggingface.py:118] Using device 'cuda'
config.json: 100%|██████████| 570/570 [00:00<00:00, 1.18MB/s]
pytorch_model.bin: 100%|██████████| 911M/911M [00:44<

bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:09<00:00, 10.19it/s]


hf (pretrained=EleutherAI/pythia-410m,revision=step100000,dtype=float), limit: None, num_fewshot: None, batch_size: 64
|    Tasks     |Version|Filter|  Metric  | Value |   |Stderr|
|--------------|-------|------|----------|------:|---|-----:|
|lambada_openai|Yaml   |none  |perplexity|11.1080|±  |0.3335|
|              |       |none  |acc       | 0.5115|±  |0.0070|



In [5]:
%%bash
source mamba_env/bin/activate
python evals/lm_harness_eval.py --model hf \
    --model_args pretrained=microsoft/phi-2,trust_remote_code=True \
    --tasks lambada_openai --device cuda --batch_size 32

2024-01-12:22:38:55,870 INFO     [utils.py:148] Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-12:22:38:55,870 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-12:22:38:56,255 INFO     [config.py:58] PyTorch version 2.1.2 available.
2024-01-12:22:39:04,415 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for gsmk boolq, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:39:04,417 WARNING  [templates.py:592] Tried instantiating `DatasetTemplates` for EleutherAI/asdiv, but no prompts found. Please ignore this warning if you are creating new prompts for this dataset.
2024-01-12:22:39:07,516 INFO     [__main__.py:184] Selected Tasks: ['lambada_openai']
2024-01-12:22:39:07,530 INFO     [huggingface.py:118] Using device 'cuda'
model.safetensors.index.json: 100%|██████████| 35.7k/35.7k [00:00<00:00, 169kB/s]
model-00001-of-00002.safetensors:

bootstrapping for stddev: perplexity


100%|██████████| 100/100 [00:10<00:00,  9.12it/s]


hf (pretrained=microsoft/phi-2,trust_remote_code=True), limit: None, num_fewshot: None, batch_size: 32
|    Tasks     |Version|Filter|  Metric  |Value |   |Stderr|
|--------------|-------|------|----------|-----:|---|-----:|
|lambada_openai|Yaml   |none  |perplexity|5.5587|±  |0.1502|
|              |       |none  |acc       |0.6270|±  |0.0067|



# Example Model Usage

In [6]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.1.2+cu121


In [1]:
def test_model(model, tokenizer):
    inputs = tokenizer("Hello, I am", return_tensors="pt")
    tokens = model.generate(**inputs)
    print(tokenizer.decode(tokens[0]))

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
torch.set_default_device("cuda")
model_name = "microsoft/phi-2"
phi = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

test_model(phi, tokenizer)

/home/ubuntu/mambafs/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA driver initialization failed, you might not have a CUDA gpu.

In [5]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer
model_name = "EleutherAI/pythia-410m-deduped"
pythia = GPTNeoXForCausalLM.from_pretrained(model_name,
                                            #revision="step3000",
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name,
  #revision="step3000",
)

test_model(pythia, tokenizer)

/home/ubuntu/mambafs/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
config.json: 100%|██████████| 570/570 [00:00<00:00, 2.23MB/s]
model.safetensors: 100%|██████████| 911M/911M [00:11<00:00, 81.2MB/s] 
tokenizer_config.json: 100%|██████████| 396/396 [00:00<00:00, 2.58MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 25.1MB/s]
special_tokens_map.json: 100%|██████████| 99.0/99.0 [00:00<00:00, 685kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello, I am a newbie to the world of programming. I am trying to create a program


In [13]:
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
import torch
from transformers import AutoTokenizer

torch.set_default_device("cuda")
model_name = "state-spaces/mamba-130m"
mamba = MambaLMHeadModel.from_pretrained(
    model_name, 
    device="cuda", 
    dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
inputs = tokenizer("Hello, I am", return_tensors="pt")
tokens = mamba.generate(inputs['input_ids'], max_length=256)
print(tokenizer.decode(tokens[0]))

Hello, I am a newbie to this. I have a problem with my app. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a button that when clicked, it will open a new window. I have a b

# Finetune on Squadv2 dataset

In [1]:
from training.train_mamba_with_pythia import *

parser = argparse.ArgumentParser()
parser.add_argument("--model", type=str, default="state-spaces/mamba-130m")
parser.add_argument("--output", type=str, default="output")
parser.add_argument("--tokenizer", type=str, default="EleutherAI/gpt-neox-20b")
parser.add_argument("--learning_rate", type=float, default=5e-4)
parser.add_argument("--batch_size", type=int, default=2)
parser.add_argument("--gradient_accumulation_steps", type=int, default=1)
parser.add_argument("--optim", type=str, default="adamw_torch")
parser.add_argument("--data_path", type=str, default="squad")
parser.add_argument("--num_epochs", type=int, default=10)
args = parser.parse_args('')

/home/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MambaLMHeadModel.from_pretrained(args.model, dtype=torch.float32, device="cuda")

tokenizer = AutoTokenizer.from_pretrained(args.tokenizer)
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
data_module = SFTDataModule(
    tokenizer=tokenizer,
    data_path=args.data_path,
    limit=1000
)

Got 0 examples, preprocess...
Tokenizing dataset...


  1%|          | 1000/87599 [00:00<01:07, 1288.36it/s]


In [4]:
trainer = MambaTrainer(
    model=model,
    train_dataset=data_module.dataset,
    tokenizer=tokenizer,
    args=TrainingArguments(
        learning_rate=args.learning_rate,
        num_train_epochs=args.num_epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        optim=args.optim,
        #evaluation_strategy="steps",
        #eval_steps=50,
        output_dir=args.output,
        save_total_limit=2,
        logging_strategy='steps',
        logging_steps=50,
        save_steps=500,
    ),
    data_collator=data_module.data_collator,
)

trainer.train()
trainer.save_model(args.output)

Step,Training Loss
50,-1.081900
100,-7.881900
150,-10.136400
200,-11.280200
250,-12.048700
300,-12.514700
350,-13.228100
400,-13.680100
450,-14.391500
500,-14.829700


Checkpoint destination directory output/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [11]:
inputs = data_module.dataset[5]
prompt, answer = tokenizer.decode(inputs['input_ids']).split('A:')
answer = 'A:'+answer
print(prompt)
inputs = tokenizer(prompt, return_tensors="pt")
tokens = trainer.pythia.generate(inputs['input_ids'].to(torch.device('cuda:0')), max_length=256)
print(tokenizer.decode(tokens[0]))
print()
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


The men's basketball team has over 1,600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 NCAA tournaments. Former player Austin Carr holds the record for most points scored in a single game of the tournament with 61. Although the team has never won the NCAA Tournament, they were named by the Helms Athletic Foundation as national champions twice. The team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending UCLA's record 88-game winning streak in 1974. The team has beaten an additional eight number-one teams, and those nine wins rank second, to UCLA's 10, all-time in wins against the top team. The team plays in newly renovated Purcell Pavilion (within the Edmund P. Joyce Center), which reopened for the beginning of the 2009–2010 season. The team is coached by Mike Brey, who, as of the 2014–15 season, his fifteenth at Notre Dame, has achieved a 332-165 record. In 2009 they were invited to the NIT, where they

/home/mamba-hidden-states/mamba_env/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 407, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [10]:
inputs = data_module.dataset[5]
prompt, answer = tokenizer.decode(inputs['input_ids']).split('A:')
answer = 'A:'+answer
print(prompt)
inputs = tokenizer(prompt, return_tensors="pt")
tokens = trainer.model.generate(inputs['input_ids'].to(torch.device('cuda:0')), max_length=256)
print(tokenizer.decode(tokens[0]))
print()
print(answer)

Beyoncé embarked on The Mrs. Carter Show World Tour on April 15 in Belgrade, Serbia; the tour included 132 dates that ran through to March 2014. It became the most successful tour of her career and one of the most-successful tours of all time. In May, Beyoncé's cover of Amy Winehouse's "Back to Black" with André 3000 on The Great Gatsby soundtrack was released. She was also honorary chair of the 2013 Met Gala. Beyoncé voiced Queen Tara in the 3D CGI animated film, Epic, released by 20th Century Fox on May 24, and recorded an original song for the film, "Rise Up", co-written with Sia.
Q: One of Beyonce's most successful tours yet was which one?

Beyoncé embarked on The Mrs. Carter Show World Tour on April 15 in Belgrade, Serbia; the tour included 132 dates that ran through to March 2014. It became the most successful tour of her career and one of the most-successful tours of all time. In May, Beyoncé's cover of Amy Winehouse's "Back to Black" with André 3000 on The Great Gatsby soundtra

In [18]:
inputs = data_module.dataset[0]
tokens = trainer.model.generate(inputs['input_ids'].unsqueeze(-1).cuda(), max_length=256)
print(tokenizer.decode(tokens[0]))

OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacty of 15.74 GiB of which 16.62 MiB is free. Process 3736616 has 2.19 GiB memory in use. Process 3790542 has 13.53 GiB memory in use. Of the allocated memory 12.41 GiB is allocated by PyTorch, and 993.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Evaluate on Squadv2

In [ ]:
%%bash
python evals/lm_harness_eval.py --model mamba \
    --model_args pretrained=state-spaces/mamba-130m \
    --tasks squadv2 --device cuda --batch_size 64

# APPENDIX

In [ ]:
import torch
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXModel
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
import sys
torch.set_default_device("cuda")


# Take in the model you want to train
model_name = "state-spaces/mamba-130m"

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

tokenizer_pythia = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m-deduped")
tokenizer_pythia.eos_token = "<|endoftext|>"
tokenizer_pythia.pad_token = tokenizer_pythia.eos_token

In [ ]:
from transformers import GPTNeoXForCausalLM, AutoTokenizer, GPTNeoXModel

pythia = GPTNeoXForCausalLM.from_pretrained(
  "EleutherAI/pythia-410m-deduped",
  #output_hidden_states=True,
  #revision="step3000",
  #cache_dir="./pythia-70m-deduped/step3000",
).to(torch.device('cuda:0'))


In [ ]:
mamba = MambaLMHeadModel.from_pretrained(
    model_name, 
    device="cuda", 
    dtype=torch.float16)

In [ ]:
# Take the user input from the command line
user_message = "Give me three steps to improve my diet, and include some evidence"#input("\n> ")

# Create a prompt
n_shot_prompting = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris"
    },
    {
        "question": "Who invented the segway?",
        "answer": "Dean Kamen"
    },
    {
        "question": "What is the fastest animal?",
        "answer": "Cheetah"
    }
]

prompt = f"You are a Trivia QA bot.\nAnswer the following question succinctly and accurately."
prompt = f"{prompt}\n\n" + "\n\n".join([f"Q: {p['question']}\nA: {p['answer']}" for p in n_shot_prompting])
prompt = f"{prompt}\n\nQ: {user_message}\nA:"

# Debug print to make sure our prompt looks good
print(prompt)

# Encode the text to token IDs
input_ids = torch.LongTensor([tokenizer.encode(prompt)]).cuda()


In [ ]:
# Generate an output sequence of tokens given the input
# "out" will contain the raw token ids as integers
out = model.generate(
    input_ids=input_ids,
    max_length=256,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
pythia_out = pythia(
    input_ids=input_ids,
    output_hidden_states=True
)

mamba_out = mamba(
    input_ids=input_ids,
    output_hidden_states=True
)

In [ ]:
teacher_loss = (pythia_out.logits.softmax(dim=2)[:,:,:50280] - mamba_out.logits.softmax(dim=2)).norm(dim=2).mean()
teacher_loss

In [ ]:
import torch.nn.functional as F
import math
mu = 0
std = math.sqrt(1.0/mamba_out.hidden_states[0].shape[-1])
size = (1, pythia_out.hidden_states[0].shape[-1], mamba_out.hidden_states[0].shape[-1])
W = torch.normal(0, std, size).to(torch.device('cuda:0'))


F.cosine_similarity(pythia_out.hidden_states[0]@ W,  mamba_out.hidden_states[0], dim=2)


In [ ]:
for k,X in enumerate(pythia_out.hidden_states):
    print(k, X.shape)
for k,X in enumerate(mamba_out.hidden_states):
    print(k, X.shape)

In [ ]:
# you must use the tokenizer to decode them back into strings
decoded = tokenizer.batch_decode(out)[0]
print("="*80)
print(decoded)
# out returns the whole sequence plus the original
cleaned = decoded.replace(prompt, "")

# the model will just keep generating, so only grab the first one
# cleaned = cleaned.split("\n\n")[0]
print(cleaned)

/home/ubuntu/test-fs2/project/mamba-hidden-states/mambaphi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Got 0 examples, preprocess...
Tokenizing dataset...


100%|██████████| 87599/87599 [01:57<00:00, 746.07it/s]


Step,Training Loss
50,0.279300
100,0.242800
150,0.301800
200,0.264300
250,0.227300
300,0.267300
350,0.249800
400,0.214700
450,0.216000
500,0.265800


KeyboardInterrupt: 